<a href="https://colab.research.google.com/github/Muhammadsulton1/ML/blob/main/VK_LAB_Sodikov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Опираясь на данную статью 
https://www.researchgate.net/publication/360410995_A_Topic_Modeling_Comparison_Between_LDA_NMF_Top2Vec_and_BERTopic_to_Demystify_Twitter_Posts
#для тематического моделирования будет использоваться BertTopic (минус данного подхода является частая генерация выбросов)

In [ ]:
 %%capture
!pip install bertopic

In [ ]:
pip install sentence-transformers

In [ ]:
from bertopic import BERTopic
import numpy as np
import pandas as pd
import os
import json
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB, BernoulliNB

In [ ]:
text = pd.read_json('/content/drive/MyDrive/data.jsonl', lines=True)

In [ ]:
text.head()

,0,1,2,3,4,5,6,7,8,9,...,691,692,693,694,695,696,697,698,699,700
0,[С 1768 года Россия ведёт войну против Османск...,[В середине января 1770 года корпус генерала Х...,"[Там русскую армию встретил враг, по жестокост...",[За зиму она унесла жизни нескольких тысяч сол...,[Вскоре эпидемия вспыхнула в близлежащей Польш...,"[Несмотря на то, что ещё весной бушующую в Мол...","[Распространение чумы продолжилось, и осенью о...",[В течение последующего года количество умерши...,"[О том, как изменилась Москва с начала эпидеми...",[Найти его уже невозможно – летом-осенью 1770 ...,...,None,None,None,None,None,None,None,None,None,None
1,[Роль одного из главных злодеев в южнокорейско...,"[Его персонаж Чан Док Су попал в игру, украв д...","[Он жесток, не боится убивать и предавать — эт...","[Хо Сон-тхэ — 43 года, он родился в октябре 19...","[По данным Today, будущий актёр изучал русский...","[Свободно владея языком, там он отвечал за про...",[На ток-шоу «Счастливы вместе» Сон-тхэ рассказ...,[Затем Сон-тхэ перешёл в крупную судостроитель...,[На этой работе его доход составлял 70 миллион...,[В 2011 году Сон-тхэ занял пятое место на шоу ...,...,None,None,None,None,None,None,None,None,None,None
2,[В СССР преуменьшали значение программы ленд-л...,[В Российском военно-историческом общества и с...,[На деле союзники оказывали Москве не только г...,"[Некоторые российские историки убеждены, что б...",[Американская и британская техника помогала об...,[А Иосиф Сталин в разговоре с британским премь...,[Больше половины всех грузов по ленд-лизу дост...,[Ещё об одном одном важном маршруте — авиатрас...,[11 марта 1941 года президент США Франклин Руз...,"[В Вашингтоне считали, что американская продук...",...,None,None,None,None,None,None,None,None,None,None
3,[В 1933 году в России началась третья в истори...,"[Мужчин, вступающих в однополые контакты, прир...","[Их подвергли репрессиям, публично обвиняя в р...",[Мужеложство было уголовно наказуемо на протяж...,"[За это время, по разным оценкам, осудили от 2...",[Среди них и известные люди – деятели культуры...,[Тем поразительнее на фоне антигомосексуальной...,[Приглашение поступило лично от высокопоставле...,[Трём представителям организации Homosexuelle ...,"[Их поселили в престижный отель «Метрополь», в...",...,None,None,None,None,None,None,None,None,None,None
4,[В январе 2021 года известный американский вед...,[За свою 63-летнюю карьеру Кинг взял тысячи ин...,[В разное время гостями его шоу были Барак Оба...,"[Его образ — подтяжки, очки и галстук — копиро...",[Ведущий начинал карьеру с разнорабочего на ра...,[Кинг прославился на американском телевидении ...,[Ларри Кинг определился с будущей профессией с...,[В подростковом возрасте ведущий часто ходил в...,[В 1942 году семья Ларри оказалась в тяжёлом ф...,[Матери мальчика пришлось содержать его и млад...,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
print("shape of text")
text.shape

shape of text


(1300, 701)

#делаем так чтобы каждый текст находился в одной ячейке

In [ ]:
data = np.array(text.values.tolist())
print('shape of data after preprocess', data.shape)

shape of data after preprocess (1300, 701)


<ipython-input-5-9344f0c84cb1>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.array(text.values.tolist())


In [ ]:
data = data.reshape(-1)

In [ ]:
df = pd.DataFrame({'text':data, 'target':range(len(data))})

In [ ]:
df.head()

,text,target
0,[С 1768 года Россия ведёт войну против Османск...,0
1,[В середине января 1770 года корпус генерала Х...,1
2,"[Там русскую армию встретил враг, по жестокост...",2
3,[За зиму она унесла жизни нескольких тысяч сол...,3
4,[Вскоре эпидемия вспыхнула в близлежащей Польш...,4


In [ ]:
print("% of nans in dataframe")
df.isna().sum() / df.shape[0]

% of nans in dataframe


text      0.87205
target    0.00000
dtype: float64

In [ ]:
df.dropna(inplace=True)

In [ ]:
print("% of nans in dataframe")
df.isna().sum() / df.shape[0]

% of nans in dataframe


text      0.0
target    0.0
dtype: float64

In [ ]:
df['text'][1]

['В середине января 1770 года корпус генерала Христофора Штофельна, призванный очистить Молдавию и Валахию от турок, разместился в столице Молдавского княжества городе Яссы.',
 0]

In [ ]:
df['target'] = df.apply(lambda x: x['text'][1], axis=1) 
df['text'] = df.apply(lambda x: x['text'][0], axis=1)

In [ ]:
df.head()

,text,target
0,С 1768 года Россия ведёт войну против Османско...,1
1,В середине января 1770 года корпус генерала Хр...,0
2,"Там русскую армию встретил враг, по жестокости...",0
3,За зиму она унесла жизни нескольких тысяч солдат.,0
4,Вскоре эпидемия вспыхнула в близлежащей Польше...,1


In [ ]:
df['text'][0]

'С 1768 года Россия ведёт войну против Османской империи.'

#препроцессинг данных

In [ ]:
def capitalizer(string: str) -> str:
    return string.lower()
def change_point(x):
  return x.replace('.','')
def change_coma(x):
  return x.replace(',','')
def remove_emails(x):
     return re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)',"", x)
def remove_urls(x):
    return re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , x)
def remove_rt(x):
    return re.sub(r'\brt\b', '', x).strip()
def remove_special_chars(x):
    x = re.sub(r'[^\w ]+', "", x)
    x = ' '.join(x.split())
    return x
def remove_digit(x):
    return ''.join([i for i in x if not i.isdigit()])

In [ ]:
df['text_clean'] = df['text'].apply(lambda x: capitalizer(x))
df['text_clean'] = df['text_clean'].apply(lambda x: change_point(x))
df['text_clean'] = df['text_clean'].apply(lambda x: change_coma(x))
df['text_clean'] = df['text_clean'].apply(lambda x: remove_emails(x))
df['text_clean'] = df['text_clean'].apply(lambda x: remove_urls(x))
df['text_clean'] = df['text_clean'].apply(lambda x: remove_rt(x))
df['text_clean'] = df['text_clean'].apply(lambda x: remove_special_chars(x))
df['text_clean'] = df['text_clean'].apply(lambda x: remove_digit(x))

In [ ]:
df.head()

,text,target,text_clean
0,С 1768 года Россия ведёт войну против Османско...,1,с года россия ведёт войну против османской им...
1,В середине января 1770 года корпус генерала Хр...,0,в середине января года корпус генерала христо...
2,"Там русскую армию встретил враг, по жестокости...",0,там русскую армию встретил враг по жестокости ...
3,За зиму она унесла жизни нескольких тысяч солдат.,0,за зиму она унесла жизни нескольких тысяч солдат
4,Вскоре эпидемия вспыхнула в близлежащей Польше...,1,вскоре эпидемия вспыхнула в близлежащей польше...


In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop_words = stopwords.words('russian')

In [ ]:
#токенизация текста
df = df.assign(text_tokenize = df['text_clean'].apply(word_tokenize))

In [ ]:
df['text_tokenize'] = df['text_tokenize'].apply(lambda x: [item for item in x if item not in stop_words])

In [ ]:
df['text_tokenize'] = df['text_tokenize'].apply(lambda x: ' '.join(x))

In [ ]:
df.head()

,text,target,text_clean,text_tokenize
0,С 1768 года Россия ведёт войну против Османско...,1,с года россия ведёт войну против османской им...,года россия ведёт войну против османской империи
1,В середине января 1770 года корпус генерала Хр...,0,в середине января года корпус генерала христо...,середине января года корпус генерала христофор...
2,"Там русскую армию встретил враг, по жестокости...",0,там русскую армию встретил враг по жестокости ...,русскую армию встретил враг жестокости уступаю...
3,За зиму она унесла жизни нескольких тысяч солдат.,0,за зиму она унесла жизни нескольких тысяч солдат,зиму унесла жизни нескольких тысяч солдат
4,Вскоре эпидемия вспыхнула в близлежащей Польше...,1,вскоре эпидемия вспыхнула в близлежащей польше...,вскоре эпидемия вспыхнула близлежащей польше а...


In [ ]:
df_nfm = df.copy()

In [ ]:
# from pymystem3 import Mystem

In [ ]:
# m = Mystem()
# df['text_tokenize'] = df['text_tokenize'].apply(lambda x: m.lemmatize(x))

In [ ]:
df.head()

,text,target,text_clean,text_tokenize
0,С 1768 года Россия ведёт войну против Османско...,1,с года россия ведёт войну против османской им...,года россия ведёт войну против османской империи
1,В середине января 1770 года корпус генерала Хр...,0,в середине января года корпус генерала христо...,середине января года корпус генерала христофор...
2,"Там русскую армию встретил враг, по жестокости...",0,там русскую армию встретил враг по жестокости ...,русскую армию встретил враг жестокости уступаю...
3,За зиму она унесла жизни нескольких тысяч солдат.,0,за зиму она унесла жизни нескольких тысяч солдат,зиму унесла жизни нескольких тысяч солдат
4,Вскоре эпидемия вспыхнула в близлежащей Польше...,1,вскоре эпидемия вспыхнула в близлежащей польше...,вскоре эпидемия вспыхнула близлежащей польше а...


In [ ]:
# df['text_tokenize'] = df['text_tokenize'].apply(lambda x: ' '.join(x))

In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distiluse-base-multilingual-cased-v1", device="cuda")
topic_model = BERTopic(embedding_model=sentence_model, verbose=True)

NameError: ignored

In [ ]:
topics, probs = topic_model.fit_transform(df['text_tokenize'].values)

In [ ]:
topic_model.get_topic_info()

In [ ]:
topic_model.get_topic(-1)

In [ ]:
topic_model.get_representative_docs(-1)

In [ ]:
topic_model.get_representative_docs(0)

In [ ]:
topic_model.update_topics(df['text_tokenize'], n_gram_range=(1, 2))

In [ ]:
topic_model.get_topic_info()

#сокращает количество топиков путем нахождения близких посмыслов друг другу топиков

In [ ]:
topic_model.reduce_topics(df['text_tokenize'], nr_topics=800)

In [ ]:
topic_model.get_topic_info()

In [ ]:
text_topic = pd.DataFrame({"topic": topics, 'text': df['text']})

In [ ]:
text_topic.tail()

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart()

#NMF используем из классических подходов

In [ ]:
from pymystem3 import Mystem

In [ ]:
df_nfm.head()

,text,target,text_clean,text_tokenize
0,С 1768 года Россия ведёт войну против Османско...,1,с года россия ведёт войну против османской им...,года россия ведёт войну против османской империи
1,В середине января 1770 года корпус генерала Хр...,0,в середине января года корпус генерала христо...,середине января года корпус генерала христофор...
2,"Там русскую армию встретил враг, по жестокости...",0,там русскую армию встретил враг по жестокости ...,русскую армию встретил враг жестокости уступаю...
3,За зиму она унесла жизни нескольких тысяч солдат.,0,за зиму она унесла жизни нескольких тысяч солдат,зиму унесла жизни нескольких тысяч солдат
4,Вскоре эпидемия вспыхнула в близлежащей Польше...,1,вскоре эпидемия вспыхнула в близлежащей польше...,вскоре эпидемия вспыхнула близлежащей польше а...


In [ ]:
m = Mystem()
df_nfm['text_tokenize'] = df_nfm['text_tokenize'].apply(lambda x: m.lemmatize(x))

In [ ]:
df_nfm['text_tokenize'] = df_nfm['text_tokenize'].apply(lambda x: ' '.join(x))

In [ ]:
df_nfm.head(3)

,text,target,text_clean,text_tokenize
0,С 1768 года Россия ведёт войну против Османско...,1,с года россия ведёт войну против османской им...,год россия вести война против османс...
1,В середине января 1770 года корпус генерала Хр...,0,в середине января года корпус генерала христо...,середина январь год корпус генерал х...
2,"Там русскую армию встретил враг, по жестокости...",0,там русскую армию встретил враг по жестокости ...,русский армия встречать враг жестокост...


In [ ]:
text_data = df_nfm.copy()

In [ ]:
vectorizer = TfidfVectorizer(max_features=1500, min_df=10, stop_words='english')
X_nfm = vectorizer.fit_transform(df_nfm['text_tokenize'])
words = np.array(vectorizer.get_feature_names_out())

print(X_nfm)
print("X = ", words)

In [ ]:
model = NMF(n_components=50, random_state=5)
model.fit(X_nfm)
nmf_features = model.transform(X_nfm)

In [ ]:
components_df = pd.DataFrame(model.components_, columns=words)
components_df

In [ ]:
for topic in range(components_df.shape[0]):
    tmp = components_df.iloc[topic]
    print(f'Топик {topic+1}, слова с высокими значениями:')
    print(tmp.nlargest(10))
    print('\n')

#предсказание для новых слов

In [ ]:
new_word = 'Я пришел с больницы и пошел гулять итп'
X = vectorizer.transform([new_word])
nmf_features = model.transform(X)
pd.DataFrame(nmf_features).idxmax(axis=1)

#Классификация на основе классического ML

In [ ]:
text_data.head()

,text,target,text_clean,text_tokenize
0,С 1768 года Россия ведёт войну против Османско...,1,с года россия ведёт войну против османской им...,года россия ведёт войну против османской империи
1,В середине января 1770 года корпус генерала Хр...,0,в середине января года корпус генерала христо...,середине января года корпус генерала христофор...
2,"Там русскую армию встретил враг, по жестокости...",0,там русскую армию встретил враг по жестокости ...,русскую армию встретил враг жестокости уступаю...
3,За зиму она унесла жизни нескольких тысяч солдат.,0,за зиму она унесла жизни нескольких тысяч солдат,зиму унесла жизни нескольких тысяч солдат
4,Вскоре эпидемия вспыхнула в близлежащей Польше...,1,вскоре эпидемия вспыхнула в близлежащей польше...,вскоре эпидемия вспыхнула близлежащей польше а...


In [ ]:
data_clf = text_data.copy()

In [ ]:
data_clf.drop(['text','text_clean'], axis=1, inplace=True)

In [ ]:
data_clf.head()

,target,text_tokenize
0,1,года россия ведёт войну против османской империи
1,0,середине января года корпус генерала христофор...
2,0,русскую армию встретил враг жестокости уступаю...
3,0,зиму унесла жизни нескольких тысяч солдат
4,1,вскоре эпидемия вспыхнула близлежащей польше а...


In [ ]:
data_clf.shape[0]

116601

In [ ]:
cv_vectorizer = TfidfVectorizer()#CountVectorizer()
cv_vectorizer.fit(data_clf['text_tokenize'])
print(len(cv_vectorizer.vocabulary_))
X = cv_vectorizer.transform(data_clf['text_tokenize'])
X = X.toarray()
feature_names = cv_vectorizer.get_feature_names()
X=pd.DataFrame(X, columns = feature_names)
X = X[:116601]
Y = data_clf['target'].values[:116601]
X_train,X_test,y_train,y_test = train_test_split(X,Y, test_size=0.25, random_state=17)

157819


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.25, random_state=17)

In [ ]:
models = [LogisticRegression(), GaussianNB(), BernoulliNB(), 
         SVC()]
score = []
for classifier in models:
  pipe = Pipeline(steps=[('scaler', StandardScaler()),
                      ('classifier', classifier)])
  pipe.fit(X_train, y_train) 
  score.append({'classifier_name':classifier, 
  'recall_score':recall_score(y_test,pipe.predict(X_test), average='macro'),
  'precision_score':precision_score(y_test,pipe.predict(X_test), average='macro'),
  'f1_score':f1_score(y_test,pipe.predict(X_test), average='macro'),
  'accuracy_score':accuracy_score(y_test,pipe.predict(X_test))})

#классификация текста при помощи берта

In [ ]:
from tqdm import tqdm
import torch
import transformers
import torch.nn as nn
from transformers import AutoModel, BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

tqdm.pandas()

device = torch.device('cuda')

In [ ]:
bert = AutoModel.from_pretrained('rubert_base_cased_sentence/')

tokenizer = BertTokenizer.from_pretrained('rubert_base_cased_sentence/')

In [ ]:
train_text = df['text_clean'].astype('str')
train_labels = df['target']
val_text = df['text_clean'].astype('str')
val_labels = df['target']
test_text = df['text_clean'].astype('str')
test_labels = df['target']

In [ ]:
seq_len = [len(str(i).split()) for i in train_text]
pd.Series(seq_len).hist(bins = 50)

In [ ]:
tokens_train = tokenizer.batch_encode_plus(
    train_text.values,
    max_length = 50,
    padding = 'max_length',
    truncation = True
)
tokens_val = tokenizer.batch_encode_plus(
    val_text.values,
    max_length = 50,
    padding = 'max_length',
    truncation = True
)
tokens_test = tokenizer.batch_encode_plus(
    test_text.values,
    max_length = 50,
    padding = 'max_length',
    truncation = True
)

train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.values)

val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.values)

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.values)
batch_size = 8


train_data = TensorDataset(train_seq, train_mask, train_y)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size = batch_size)

val_data =  TensorDataset(val_seq, val_mask, val_y)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size = batch_size)

In [ ]:
for param in bert.parameters():
    param.requires_grad = False

class BERT_Arch(nn.Module):
    
    def __init__(self, bert):
        super(BERT_Arch, self).__init__()
        self.bert = bert
        self.dropout = nn.Dropout(0.1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(768,512)
        self.fc2 = nn.Linear(512, 3)
        self.softmax = nn.LogSoftmax(dim = 1)
    
    def forward(self, sent_id, mask):
        _, cls_hs = self.bert(sent_id, attention_mask = mask, return_dict = False)
        x = self.fc1(cls_hs)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

In [ ]:
model = BERT_Arch(bert)

model = model.to(device)
from transformers import AdamW

optimizer = AdamW(model.parameters(),
               lr= 1e-3)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', np.unique(train_labels), train_labels)

print(class_weights)


weights = torch.tensor(class_weights, dtype = torch.float)
weights = weights.to(device)
cross_entropy = nn.CrossEntropyLoss()
epochs = 20

In [ ]:
def train():
    model.train()
    total_loss, total_accuracy = 0, 0
    total_preds = []
    
    for step, batch in tqdm(enumerate(train_dataloader), total = len(train_dataloader)):
        batch = [r.to(device) for r in batch]
        sent_id,mask,labels = batch
        model.zero_grad()
        preds = model(sent_id, mask)
        loss = cross_entropy(preds, labels)
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        preds = preds.detach().cpu().numpy()
        total_preds.append(preds)
        
    avg_loss = total_loss / len(train_dataloader)
    total_preds = np.concatenate(total_preds, axis = 0)
    
    return avg_loss, total_preds

In [ ]:
def evaluate():
    model.eval()
    total_loss, total_accuracy = 0,0
    total_preds = []

    for step, batch in tqdm(enumerate(val_dataloader), total = len(val_dataloader)):
        batch = [t.to(device) for t in batch]
        sent_id, mask, labels = batch
        
        with torch.no_grad():
            preds = model(sent_id, mask)
            loss = cross_entropy(preds, labels)
            total_loss = total_loss + loss.item()
            preds = preds.detach().cpu().numpy()
            total_preds.append(preds)

    avg_loss = total_loss / len(val_dataloader)
    total_preds = np.concatenate(total_preds, axis = 0)

In [ ]:
best_valid_loss = float('inf')

train_losses = []
valid_losses = []

for epoch in range(epochs):
    print('\n Epoch{:} / {:}'.format(epoch+1, epochs))
    
    train_loss, _ = train()
    valid_loss, _ = evaluate()
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    print(f'\nTraining loss: {train_loss:.3f}')
    print(f'Validation loss: {valid_loss:.3f}')

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

list_seq = np.array_split(test_seq, 50)
list_mask = np.array_split(test_mask, 50)


predictions = []
for num, elem in enumerate(list_seq):
    with torch.no_grad():
        preds = model(elem.to(device), list_mask[num].to(device))
        predictions.append(preds.detach().cpu().numpy())

In [ ]:
flat_preds = [item[1] for sublist in predictions for item in sublist]
flat_preds = (flat_preds - min(flat_preds)) / (max(flat_preds) - min(flat_preds))
df['confidence'] = flat_preds

In [ ]:
df['pred'] = df['confidence'].apply(lambda x: 1 if x>0.92 else 0)

print(classification_report(df['target'], df['pred']))